In [1]:
from pypylon import pylon
import cv2
from PIL import Image
import os
import numpy as np
from pyzbar.pyzbar import decode

In [2]:
def find_barcode_orientation(image_path):
    # Read the image
    image = cv2.imread(image_path)

    # Detect edges
    edges = cv2.Canny(image, 50, 150, apertureSize=3)

    # Detect lines
    lines = cv2.HoughLines(edges, 1, np.pi / 180, 100)

    # Check if lines are detected
    if lines is not None and len(lines) > 0:
        # Take the first line and calculate the angle
        rho, theta = lines[0][0]
        angle = theta * 180 / np.pi

        # Print the angle
        print(f"Barkodun açısı: {angle} derece")

        # Rotate the image
        rotated_image = rotate_image(image, angle)

        # Save the rotated image
        load_path = r"C:\Users\Admin\Desktop\Barkodv1\images\2.png"
        cv2.imwrite(load_path, rotated_image)
        print("Rotated image saved as 'rotated_image.png'")
    else:
        print("No lines detected in the image.")

def rotate_image(image, angle):
    # Adjust the angle if necessary
    if angle < 0:
        angle += 180

    # Rotate the image
    rows, cols, _ = image.shape
    rotation_matrix = cv2.getRotationMatrix2D((cols / 2, rows / 2), angle, 1)

    # Determine the new dimensions of the rotated image
    cos = np.abs(rotation_matrix[0, 0])
    sin = np.abs(rotation_matrix[0, 1])
    new_width = int((rows * sin) + (cols * cos))
    new_height = int((rows * cos) + (cols * sin))

    # Adjust the rotation matrix to take into account the translation
    rotation_matrix[0, 2] += (new_width - cols) / 2
    rotation_matrix[1, 2] += (new_height - rows) / 2

    # Rotate and resize the image
    rotated_image = cv2.warpAffine(image, rotation_matrix, (new_width, new_height))
    return rotated_image

In [3]:
def read_barcodes(image_path):
    # Resmi oku
    image = cv2.imread(image_path)

    # Barkodları ve QR kodlarını tespit et
    decoded_objects = decode(image)

    # Her tespit edilen nesne için bilgileri ekrana yazdır
    for obj in decoded_objects:
        barcode_data = obj.data.decode('utf-8')
        print(f'Tür: {obj.type}, Veri: {barcode_data}')

        # Nesnenin konumunu çerçevele
        points = obj.polygon
        if len(points) >= 4:
            hull = cv2.convexHull(np.array(points, dtype=np.int32))  # numpy dizisine dönüştür
            cv2.polylines(image, [hull], True, (0, 255, 0), 2)

   # Pencereyi boyutlandır (örneğin, 800x600)
    cv2.namedWindow('Barcode Reader', cv2.WINDOW_NORMAL)
    cv2.resizeWindow('Barcode Reader', 800, 600)

    cv2.imshow('Barcode Reader', image)

    # "l" tuşuna basılmasını bekle (0 ms bekletme süresi)
    key = cv2.waitKey(0)

    # "l" tuşuna basıldığında sadece "Barcode Reader" penceresini kapat
    if key == ord('l'):
        cv2.destroyWindow('Barcode Reader')




In [4]:
def decode_custom_barcodes(image):
    # Özel barkod türleri için decoder kullanarak tespit et
    decoded_dmtx = dmtx_decode(image)
    decoded_pdf417 = pdf417_decode(image)

    # Dönen sonuçları birleştir
    decoded_objects = decoded_dmtx + decoded_pdf417

    return decoded_objects

In [5]:
# Pylon sürücüsünü başlat
pylon.tl_factory_instantiator = pylon.TlFactory.GetInstance
tl_factory = pylon.TlFactory.GetInstance()

# Bağlı kamera cihazlarını al
devices = tl_factory.EnumerateDevices()
if not devices:
    print("Bağlı kamera bulunamadı.")
    exit()

# İlk kamerayı seç
camera = pylon.InstantCamera(tl_factory.CreateDevice(devices[0]))

# Kamerayı aç
camera.Open()

# Video penceresini aç
cv2.namedWindow("Pylon Video", cv2.WINDOW_NORMAL)

# Yeni pencere için
cv2.namedWindow("Etiket Görüntüsü", cv2.WINDOW_NORMAL)

# Dosya yolu ve isim
file_path = r"C:\Users\Admin\Desktop\Barkodv1\images\1.png"
# Dosya yolu içindeki dizinleri oluştur
os.makedirs(os.path.dirname(file_path), exist_ok=True)


try:
    while True:
        grab_result = camera.GrabOne(500)
        if grab_result.GrabSucceeded():
            image = grab_result.Array.astype('float32') / 255.0
            _, binary_image = cv2.threshold(image, 0.25, 1.0, cv2.THRESH_BINARY)

            # Konturları bul
            contours, _ = cv2.findContours((binary_image * 255).astype('uint8'), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

            # Kontur bulunamadıysa devam et
            if not contours:
                continue

            # En büyük konturu bul
            largest_contour = max(contours, key=cv2.contourArea)

            # Etiketi çevreleyen dikdörtgeni al
            x, y, w, h = cv2.boundingRect(largest_contour)

            # Dikdörtgeni biraz genişlet
            margin = 50  # Eklenecek/çıkartılacak kenar pikselleri sayısı
            x -= margin
            y -= margin
            w += 2 * margin
            h += 2 * margin

            # Koordinatlar negatif değer alabilir, kontrol et
            x = max(x, 0)
            y = max(y, 0)

            # Dikdörtgeni çizmek için bir kopya oluştur
            rectangle_image = binary_image.copy()

            # Etiketi içeren dikdörtgeni çiz
            cv2.rectangle(rectangle_image, (x, y), (x + w, y + h), (1.0, 0, 0), 2)

            # Görüntüyü göster
            cv2.imshow("Pylon Video", rectangle_image)

            # Etiket görüntüsünü al
            label_image = image[y:y+h, x:x+w]

            # Etiket görüntüsünü yeni pencerede göster
            cv2.imshow("Etiket Görüntüsü", label_image)

            key = cv2.waitKey(1)

            if key == ord('k'):  # "k" tuşuna basıldığında
                file_path = r"C:\Users\Admin\Desktop\Barkodv1\images\1.png"
                image_path = r'C:\Users\Admin\Desktop\Barkodv1\images\2.png'
                pil_image = Image.fromarray((label_image * 255).astype('uint8'))
                pil_image.save(file_path)
                print(f"Görüntü kaydedildi: {file_path}")
                find_barcode_orientation(file_path)
                read_barcodes(image_path)
                
                continue  # Programın devam etmesine izin ver

            elif key == ord('q'):  # "q" tuşuna basıldığında
                break

except KeyboardInterrupt:
    camera.Close()
    cv2.destroyAllWindows()
    print("Program kapatıldı.")

Görüntü kaydedildi: C:\Users\Admin\Desktop\Barkodv1\images\1.png
Barkodun açısı: 4.99999985454646 derece
Rotated image saved as 'rotated_image.png'
Tür: CODE128, Veri: P210586.U1-R053
Tür: CODE128, Veri: 3000
Tür: CODE128, Veri: 2305-P210586.U1
Tür: CODE128, Veri: 1N4148W-7-F
Görüntü kaydedildi: C:\Users\Admin\Desktop\Barkodv1\images\1.png
Barkodun açısı: 120.99999767530745 derece
Rotated image saved as 'rotated_image.png'
Tür: CODE128, Veri: 9D2225
Tür: CODE128, Veri: Q5000
Görüntü kaydedildi: C:\Users\Admin\Desktop\Barkodv1\images\1.png
Barkodun açısı: 106.000000672989 derece
Rotated image saved as 'rotated_image.png'
Tür: CODE128, Veri: 1PRC0603FR-07100KL
Tür: CODE128, Veri: 9D2225
Tür: CODE128, Veri: Q5000
Görüntü kaydedildi: C:\Users\Admin\Desktop\Barkodv1\images\1.png
Barkodun açısı: 171.99999567941717 derece
Rotated image saved as 'rotated_image.png'
Tür: CODE128, Veri: 2305-P210586.U1
Tür: CODE128, Veri: 3000
Tür: CODE128, Veri: P210586.U1-R053
Tür: CODE128, Veri: 1N4148W-7-F
G

RuntimeException: Failed to open stream grabber: False. cam:(p=29711, add=3.3.168.192) adapter:(p=0, add=2.3.168.192) multi:(nullptr) (0x21000001) : RuntimeException thrown (file 'gxstream.cpp', line 408)

### https://pypi.org/project/pdf417decoder/

### https://pypi.org/project/pylibdmtx/